In [ ]:
BASE_DIR = "/Users/denis/MyProjects/Summarization"
GRAPH_BASE_DIR = "/Users/denis/MyProjects/uncased_L-12_H-768_A-12"

In [ ]:
import os
from tree_rank_summaraiser import TreeRankSummariser
import global_const
import pandas as pd
import glob
import time
import logging
import numpy as np
logging.basicConfig(level=logging.DEBUG)
from encoders.encoder_factory import EncoderFactory


In [ ]:
GRAPH_PATH = os.path.join(GRAPH_BASE_DIR, "model_for_inference.pbtxt")
VOCAB_PATH = os.path.join(GRAPH_BASE_DIR, "vocab.txt")

encoder_conf_dict = {
    "default": {
        "type": "bert_embedded",
        "input_dim": 1,
        "output_dim": 768,
        "params": {
            "graph": {
                "path_desc": {
                    "type": "absolute",
                    "file": GRAPH_PATH
                }
            },
            "vocab": {
                "path_desc": {
                    "type": "absolute",
                    "file": VOCAB_PATH
                }
            }
        }
   }
}
encoder_factory = EncoderFactory(encoder_conf_dict)

encoder = encoder_factory.get_encoder("default")

In [ ]:
summariser = TreeRankSummariser(3, encoder)

In [ ]:
pattern_str = "/Users/denis/MyProjects/Summarization/data/Companies' news/Norwegian Cruise Line/*"
files_list = glob.glob(pattern_str)
data_list = []
company_name_list = ["Norwegian Cruise Line", "Norwegian Cruise"]
# company_name_list = ["Carnival"]
# company_name_list = ["Pfizer"]

companies = [
    {
        global_const.COMPANY_ISIN: "Norwegian Cruise Line",
        global_const.COMPANY_NAMES: ["Norwegian Cruise Line", "Norwegian Cruise"]
     },
    {
        global_const.COMPANY_ISIN: "Carnival corporation",
        global_const.COMPANY_NAMES: ["carnival corporation", "ccl"]
    }
]

time_list = []
for path in files_list:
    print(path)
    with open(path) as file:
        article_str = file.read()
    
    start = time.time()
    article_sum_dict = summariser.get_summary_for_companies(article_str, companies)
    finish = time.time()
    time_list.append(finish - start)
    for isin, summary in article_sum_dict.items():
        data_list.append({
            "News": article_str,
            "Summarization": summary,
            "isin": isin
        })
    
pd.DataFrame(data=data_list).to_excel(f"/Users/denis/MyProjects/Summarization/data/Companies' news/Norwegian Cruise Line.xls")

In [ ]:
time_np = np.array(time_list)

In [ ]:
time_np.mean()

In [ ]:
np.median(time_np)

In [ ]:
time_np